In [1]:
import os
import time
import requests

ELEFAN_PRODUCT_API_URL = os.environ.get("API_URL")
OFF_API_URL = "https://fr.openfoodfacts.org/api/v2/product"

ELEFAN_FOOD_FAMILLE_CODE_TO_EXCLUDE = [
    21,  # bebe (inclus nourriture...)
    22,  # hygiène, beauté
    23,  # entretien, nettoyage
    24,  # cuisine
    25,  # santé
    31,  # journaux
    41,  # maison
    42,  # papeterie
    43,  # textile
    44,  # animalerie
    45,  # jardinerie
    90,  # jeux
]

In [4]:
"""
Récupérer les produits depuis Elefan (via l'API)
"""
print("Elefan Grenoble")
elefan_product_list = list()

response = requests.get(ELEFAN_PRODUCT_API_URL)
elefan_product_list = response.json()
print("Nombre d'articles (dans KASO) :", len(elefan_product_list))
# print(elefan_product_list[0])

elefan_product_actifs_list = [p for p in elefan_product_list if p['status'] == 'ACTIF']
print("Nombre d'articles actifs (status = ACTIF dans KASO) :", len(elefan_product_actifs_list))

elefan_product_actifs_ean_list = [p for p in elefan_product_actifs_list if (len(str(p['code'])) in [8, 13])]
print("Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) :", len(elefan_product_actifs_ean_list))

elefan_product_actifs_ean_filtered_list = [p for p in elefan_product_actifs_ean_list if p['famille']['code'] not in ELEFAN_FOOD_FAMILLE_CODE_TO_EXCLUDE]
print("Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) :", len(elefan_product_actifs_ean_filtered_list))

Elefan Grenoble
Nombre d'articles (dans KASO) : 2106
Nombre d'articles actifs (status = ACTIF dans KASO) : 1456
Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) : 1124
Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) : 758


In [5]:
print(f"Lien avec OFF (pour les {len(elefan_product_actifs_ean_filtered_list)} articles filtrés)")
found = 0

for index, elefan_product in enumerate(elefan_product_actifs_ean_filtered_list):
    response = requests.get(f"{OFF_API_URL}/{elefan_product['code']}.json")
    if response.status_code == 200:
        found += 1
        elefan_product_actifs_ean_filtered_list[index]['OFF'] = response.json()
    if (index % 10) == 0:
        time.sleep(1)
    if (index % 100) == 0:
        print(f"progress: {index}...")

print("Trouvés", found, "/", len(elefan_product_actifs_ean_filtered_list))

Lien avec OFF (pour les 758 articles filtrés)
progress: 0...
progress: 100...
progress: 200...
progress: 300...
progress: 400...
progress: 500...
progress: 600...
progress: 700...
Trouvés 529 / 758


In [7]:
from collections import Counter

elefan_product_actifs_ean_filtered_off_list = [p for p in elefan_product_actifs_ean_filtered_list if 'OFF' in p]
print(f"Sur les {len(elefan_product_actifs_ean_filtered_off_list)} produits trouvés dans OFF...")

for key in ["product_name", "quantity", "image_url", "categories", "brands", "labels", "origins", "packaging"]:
    print(f"Avec {key} :", len([p for p in elefan_product_actifs_ean_filtered_off_list if (key in p['OFF']['product'] and p['OFF']['product'][key])]))

for key in ["nutriscore_grade", "ecoscore_grade"]:
    key_list = list()
    for p in elefan_product_actifs_ean_filtered_off_list:
        if key in p['OFF']['product']:
            key_list.append(p['OFF']['product'][key])
    print(f"{key} :", Counter(key_list))

for key in ["nova_groups_tags"]:  # always length 1
    key_list = list()
    for p in elefan_product_actifs_ean_filtered_off_list:
        if key in p['OFF']['product']:
            key_list.extend(p['OFF']['product'][key])
    print(f"{key} :", Counter(key_list))

Sur les 529 produits trouvés dans OFF...
Avec product_name : 523
Avec quantity : 374
Avec image_url : 512
Avec categories : 429
Avec brands : 443
Avec labels : 432
Avec origins : 151
Avec packaging : 254
nutriscore_grade : Counter({'unknown': 164, 'd': 85, 'a': 81, 'c': 70, 'b': 48, 'e': 46, 'not-applicable': 34})
ecoscore_grade : Counter({'unknown': 201, 'b': 124, 'c': 76, 'a': 71, 'd': 35, 'e': 17, 'not-applicable': 5})
nova_groups_tags : Counter({'unknown': 218, 'en:3-processed-foods': 143, 'en:4-ultra-processed-food-and-drink-products': 94, 'en:1-unprocessed-or-minimally-processed-foods': 60, 'en:2-processed-culinary-ingredients': 14})
